In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import astral as ast
from astral import sun

In [6]:
# First read the clean initial loon data

#['N139LB', 'N166LB', 'N211LB', 'N221LB', 'N225LB', 'N226LB', 'N235LB', 'N238LB', 'N252LB',\
# 'N271LB', 'N329LB', 'N789LB']

raw_df = pd.DataFrame()

flights_to_add = ['N139LB', 'N166LB', 'N191LB', 'N211LB', 'N221LB', 'N225LB', 'N226LB', 'N228LB', 'N234LB',
                  'N235LB', 'N238LB', 'N252LB', 'N253LB', 'N271LB', 'N329LB', 'N789LB']

#min_pts_list = ['1416']

for loon in flights_to_add:

    
    aux = pd.read_csv('../../../../og_data/' +  loon  +  '.csv', parse_dates=['ts'], low_memory=False)
    
   # Compute the local change in altitude
    aux = aux.sort_values('ts')

    aux['alt_change'] = aux['alt'] -  aux['alt'].shift()
    aux['alt_change'] = aux['alt_change'].fillna(0)

    raw_df = raw_df.append(aux)


In [7]:
bl = raw_df

In [8]:
# We take a look at the time zone info: the value after the '+' sign on the ts",
pd.DataFrame(bl.ts.astype(str).str.split('+').tolist(), columns = ['local_time','time_zone'])\
                            .time_zone.value_counts()


00:00    2412137
Name: time_zone, dtype: int64

In [9]:
# Since the measures are all relative to the time zone we do not convert them

def compute_day_night(lat, lon, alt, ts):

    obs = ast.Observer(lat, lon, alt)
    sun_at = ast.sun.sun(obs, ts) # UTC for time by default

    if (sun_at.get('sunrise') < ts) & (ts < sun_at.get('sunset')):
            return True

    return False

bl['sun_light'] = bl.apply(lambda r: compute_day_night(r.lat, r.lon, r.alt, r.ts), axis=1)


In [10]:
bl

,ts,aircraft_id,alt,speed,Direction,registration_id,lat,lon,alt_change,sun_light
41950,2017-12-06 01:26:44+00:00,HBAL048,40025,32,141,N139LB,40.535568,-117.813049,0.0,False
41951,2017-12-06 01:26:51+00:00,HBAL048,40050,32,139,N139LB,40.534882,-117.812256,25.0,False
41952,2017-12-06 01:26:57+00:00,HBAL048,40100,33,136,N139LB,40.534149,-117.811394,50.0,False
41953,2017-12-06 01:27:03+00:00,HBAL048,40125,33,135,N139LB,40.533554,-117.810608,25.0,False
41954,2017-12-06 01:28:09+00:00,HBAL048,40300,35,137,N139LB,40.525818,-117.800964,175.0,False
...,...,...,...,...,...,...,...,...,...,...
118324,2020-05-04 08:29:42+00:00,HBAL043,61400,18,288,N789LB,-3.685401,-83.218498,0.0,False
118325,2020-05-04 08:36:10+00:00,HBAL043,61400,17,300,N789LB,-3.671482,-83.247719,0.0,False
118326,2020-05-04 08:37:46+00:00,HBAL043,61400,17,296,N789LB,-3.667804,-83.254539,0.0,False
22333,2020-05-04 09:31:52+00:00,HBAL043,61300,17,276,N789LB,-3.589142,-83.494980,-100.0,False


In [11]:
bl.to_csv(path_or_buf='../../../../og_data/DayNight/loons_DBSCAN_with_day_night.csv', \
                    index=False, encoding='utf-8-sig')